# TP : Classification de Texte avec Scikit-Learn

Dans ce TP, nous allons explorer les outils principaux de Scikit-Learn pour une tâche de classification de documents textuels, en utilisant le dataset des *Twenty Newsgroups*.

Objectifs :

- Charger et préparer les données
- Extraire des caractéristiques des textes
- Entraîner un modèle de classification
- Évaluer les performances du modèle

Nous aborderons aussi la recherche de paramètres optimaux via une recherche en grille.


### Préparation

Pour commencer, assurez-vous d'avoir installé *scikit-learn* et les autres dépendances nécessaires.

Commençons par charger les données et définir les catégories que nous utiliserons dans cet exercice.

> Question 1 : Importez les bibliothèques nécessaires et configurez le dataset pour contenir les catégories `alt.atheism`, `soc.religion.christian`, `comp.graphics`, et `sci.med`.

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ["alt.atheism", "soc.religion.christian", "comp.graphics", "sci.med"]

dataset = fetch_20newsgroups(categories=categories, shuffle=True, random_state=13)

### Analyse des Données Chargées

Les données sont chargées sous forme d'un *bunch*, un objet contenant plusieurs attributs utiles pour notre analyse.

> Question 2 : Affichez le nombre de documents et les catégories de nouvelles chargées dans le dataset.

In [4]:
print(len(dataset.data))
print(dataset.target_names)

2257
['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


### Exploration des Données Textuelles

> Question 3 : Affichez les premières lignes du premier document, ainsi que sa catégorie associée.

Cela nous donnera un aperçu des documents et de leur contenu.

In [9]:
dataset.data[0].split("\n")[:20]

['From: geb@cs.pitt.edu (Gordon Banks)',
 'Subject: Re: Update (Help!) [was "What is This [Is it Lyme\'s?]"]',
 'Article-I.D.: pitt.19436',
 'Reply-To: geb@cs.pitt.edu (Gordon Banks)',
 'Organization: Univ. of Pittsburgh Computer Science',
 'Lines: 42',
 '',
 "In article <1993Mar29.181958.3224@equator.com> jod@equator.com (John Setel O'Donnell) writes:",
 '>',
 ">I shouldn't have to be posting here.  Physicians should know the Lyme",
 ">literature beyond Steere & co's denial merry-go-round.  Patients",
 '>should get correctly diagnosed and treated.',
 '>',
 '',
 "Why do you think Steere is doing this?  Isn't he acting in good faith?",
 'After all, as the "discoverer" of Lyme for all intents and purposes,',
 "the more famous Lyme gets, the more famous Steere gets.  I don't",
 'see the ulterior motive here.  It is easy for me to see it the',
 'those physicians who call everything lyme and treat everything.',
 'There is a lot of money involved.']

### Extraction des Caractéristiques des Textes

Pour entraîner un modèle, nous devons transformer les documents textuels en vecteurs de caractéristiques numériques.

> Question 4 : Utilisez la méthode `CountVectorizer` pour transformer les textes en une matrice de fréquences de mots, puis affichez la taille de cette matrice.

Cette matrice, de type "sac de mots", représente chaque document par les occurrences des mots qu'il contient.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dataset.data)

In [11]:
X_train_counts.shape

(2257, 35788)

### Passage aux Fréquences Relatives (tf-idf)

Pour équilibrer l'importance des mots entre documents de différentes longueurs, on utilise la fréquence de termes (TF) ou bien TF-IDF.

> Question 5 : Transformez la matrice en représentation TF-IDF et affichez la dimension de cette nouvelle matrice.


In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [18]:
X_train_tfidf.shape

(2257, 35788)

### Entraînement d'un Classifieur Naïve Bayes

Avec notre matrice de caractéristiques prête, entraînons un classifieur Naïve Bayes pour prédire la catégorie de chaque document.

> Question 6 : Entraînez un classifieur `MultinomialNB` sur le jeu de données transformé en TF-IDF.


In [23]:
from sklearn.naive_bayes import MultinomialNB

clf_nb = MultinomialNB()

clf_nb.fit(X_train_tfidf, dataset.target)

MultinomialNB()

### Prédiction sur des Nouveaux Documents

> Question 7 : Prédisez les catégories pour les phrases `"God is love"` et `"OpenGL on the GPU is fast"`.

Affichez la catégorie prédite pour chaque document.


In [ ]:
docs =  ["God is love", "OpenGL on the GPU is fast"]
X_docs_tfidf = tfidf_transformer.transform(
    count_vect.transform(docs)
)

y_pred_sample = clf_nb.predict(X_docs_tfidf)

In [25]:
print(y_pre_sample)

[3 1]


In [27]:
dataset.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

### Construction d'un Pipeline pour Simplifier le Workflow

> Question 8 : Créez un `Pipeline` qui enchaîne `CountVectorizer`, `TfidfTransformer` et `MultinomialNB`, puis entraînez le modèle sur le jeu de données.

Cela rend le processus d'extraction de caractéristiques et d'entraînement plus facile et modulaire.


In [28]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

pipeline_clf = Pipeline(steps=[
    ("vect", TfidfVectorizer()),
    ("clf", MultinomialNB()),
])
pipeline_clf.fit(dataset.data, dataset.target)

Pipeline(steps=[('vect', TfidfVectorizer()), ('clf', MultinomialNB())])

### Évaluation des Performances du Modèle

> Question 9 : Chargez le sous-ensemble de test et évaluez les performances du modèle en termes de précision moyenne.

Affichez également le rapport de classification et la matrice de confusion pour une évaluation plus détaillée.


In [29]:
dataset_test = fetch_20newsgroups(subset="test", categories=categories, shuffle=True, random_state=13)
X_test = dataset_test.data
y_test = dataset_test.target
y_pred = pipeline_clf.predict(X_test)

In [30]:
import numpy as np
from sklearn import metrics

print("Précision moyenne :", np.mean(y_pred == y_test))
print(metrics.classification_report(y_test, y_pred, target_names=dataset_test.target_names))
print(metrics.confusion_matrix(y_test, y_pred))

Précision moyenne : 0.8348868175765646
                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502

[[192   2   6 119]
 [  2 347   4  36]
 [  2  11 322  61]
 [  2   2   1 393]]


### Optimisation des Paramètres avec Grid Search

Scikit-Learn permet d'optimiser les paramètres des composants du pipeline par une recherche en grille.

> Question 10 : Utilisez `GridSearchCV` pour trouver les meilleurs paramètres parmi différents modèles de machine learning.

Testez les modèles `MultinomialNB` et `SGD`, avec une recherche de paramètres pour le modèle `SGD`.


In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

parameters = {
    "clf": [MultinomialNB(), SGDClassifier(max_iter=500, tol=1e-3)],
    "clf__alpha": (1e-2, 1e-3)
}

gridsearch_clf = GridSearchCV(pipeline_clf, parameters, cv=5)
gridsearch_clf.fit(dataset.data, dataset.target)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf': [MultinomialNB(alpha=0.01),
                                 SGDClassifier(max_iter=500)],
                         'clf__alpha': (0.01, 0.001)})

> Question 11 : Affichez les meilleurs paramètres trouvés et la précision obtenue pour ces paramètres optimaux.


In [33]:
print(gridsearch_clf.best_params_)
print(gridsearch_clf.best_score_)

{'clf': MultinomialNB(alpha=0.01), 'clf__alpha': 0.01}
0.9756313403842002


 🎉